In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from requests.exceptions import HTTPError
import sys
import math

In [11]:
url = "http://www.nepalstock.com/"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'lxml')
total_transactions = soup.find(text="Total Transactions")
total_transactions = total_transactions.parent.parent.next_sibling.next_sibling.text
total_transactions = total_transactions.replace(",", "")
total_iteration = math.ceil(int(total_transactions)/500)
print(total_iteration)
print(request.text)


173

<!DOCTYPE HTML>
<html>
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1"/>
        <title>Nepal Stock Exchange Ltd.</title>
        <meta name="title" content="NEPSE Official Website" />
        <meta content="This is NEPSE Official Website" http-equiv="description" name="description" />
        <meta content="This is NEPSE Official Website" http-equiv="keywords" name="keywords" />
        <link rel="shortcut icon" href="/favicon.ico" type="image/x-icon">
        <link rel="icon" href="/favicon.ico" type="image/x-icon">
        <!-- Bootstrap core CSS -->
        <link href="http://www.nepalstock.com/css/reset.css" rel="stylesheet">
        <link href="http://www.nepalstock.com/css/bootstrap.css" rel="stylesheet">

        <link rel="stylesheet" type="text/css" href="http://www.nepalstock.com/js/ui/theme/orange.css" />
        <!-- Custom styles for this template -->


In [30]:

table_list = list()
subgroup = "floorsheet"
# url = "http://iloveyou:8000"
r_list = list()
mistake_count = 0
for index in range(1,total_iteration+1):
    url = f"http://www.nepalstock.com/main/floorsheet/index/{index}/?contract-no=&stock-symbol=&buyer=&seller=&_limit=500/"
    while True:
        try:
            r = requests.get(url)
            if r.status_code == 200 :  
                break
        except requests.exceptions.RequestException as e:  # This is the correct syntax
            mistake_count += 1
            if mistake_count > 3:
                print(e)
                sys.exit()
                continue
        
    r_list.append(r)
    
    


[<Response [200]>]

In [39]:

for r in r_list:
    soup = BeautifulSoup(r.text, "lxml")
    table = soup.find("table", class_="table my-table").contents
    table = list(filter(lambda a : a!='\n', table))
    # row = [x.text for x in table[2].find_all("td")]
    # print(row)
    
    for row in table[2:-3]:
        datas = row.find_all("td")
        row = [x.text for x in datas]
        row[5:] = [float(x) for x in row[5:]]
        table_list.append(row)

    # data_json = json.dumps(table_list)
    # print(data_json)
columns = ['S.N.', 'Contract No','Stock Symbol', 'Buyer Broker',
           'Seller Broker', 'Quantity', 'Rate', 'Amount']
df = pd.DataFrame(table_list, columns=columns)
df.to_csv("floorsheet.csv")



In [2]:


df = pd.read_csv('./datas/floorsheet_2021-05-13_3.csv')
df


,Unnamed: 0,S.N.,Contract No,Stock Symbol,Buyer Broker,Seller Broker,Quantity,Rate,Amount
0,0,1,2021051303033167,LLBS,33,5,50.0,2369.0,118450.0
1,1,2,2021051304036380,SICL,33,38,50.0,1935.0,96750.0
2,2,3,2021051304036379,PICL,38,34,10.0,780.0,7800.0
3,3,4,2021051304036378,NGPL,38,1,50.0,317.0,15850.0
4,4,5,2021051304036377,TRH,38,3,50.0,284.0,14200.0
...,...,...,...,...,...,...,...,...,...
86291,86291,86292,2021051301000001,NIB,28,38,20.0,462.0,9240.0
86292,86292,86293,2021051303000001,DDBL,49,48,100.0,1850.0,185000.0
86293,86293,86294,2021051304000001,NHPC,57,48,10.0,168.0,1680.0
86294,86294,86295,2021051305000002,NICLBSL,8,57,5.0,1750.0,8750.0


In [3]:

# df.tail(10).style.hide_index()
df[(df["Amount"] > 100000) & (df["Buyer Broker"] == "58")].sort_values("Amount", ascending =False)
# df[(df["Amount"] > 1000000)].sort_values("Amount", ascending =False)

,Unnamed: 0,S.N.,Contract No,Stock Symbol,Buyer Broker,Seller Broker,Quantity,Rate,Amount
14559,14559,14560,2021051301011905,SHIVM,58,26,9702.0,1515.0,14698530.0
3063,3063,3064,2021051303032098,NRIC,58,6,6000.0,1584.0,9504000.0
35899,35899,35900,2021051303018178,SABSL,58,3,4256.0,1850.0,7873600.0
11110,11110,11111,2021051301012547,CCBL,58,38,25232.0,245.0,6181840.0
53794,53794,53795,2021051304016349,LEMF,58,28,499000.0,11.6,5788400.0
...,...,...,...,...,...,...,...,...,...
67211,67211,67212,2021051304009758,AKJCL,58,22,500.0,202.0,101000.0
57645,57645,57646,2021051304014498,JOSHI,58,48,500.0,202.0,101000.0
27177,27177,27178,2021051301009528,NCCB,58,45,273.0,369.0,100737.0
1561,1561,1562,2021051303032638,ICFC,58,20,300.0,335.0,100500.0
